# Interacting with the ESCALATE REST API

This is a tutorail for using the ESCALATE REST API. 

It presumes only knowledge of basic python. 

It also presumes you have a local instance of ESCALATE running at http://localhost:8000

In [1]:
import json
from pprint import pprint

import requests  # requests library will send and receive data from the escalate server
from requests.api import post

import pandas as pd

In [2]:
LANL_PROXY = 'http://proxyout.lanl.gov:8080'
proxies={'http':LANL_PROXY, 'https': LANL_PROXY}

## Quick intro to REST APIs

All you need to know about REST is that:
* It is a protocol for exchanging data between a client (e.g. me) and a server (e.g. ESCALATE)
* Data formats are human *and* machine readable (XML or JSON)
* There are two main HTTP 'verbs' or functions: 
  1. GET data from the server
  2. POST data to the server  
     (there are others, e.g. PUT and PATCH, but we won't use these in this tutorial)
* The python `requests` library implements the HTTP verbs for interacting with servers, including with REST APIs

### Example: GET from the PubChem API

* [Pubchem](https://pubchem.ncbi.nlm.nih.gov/) is a great way to get chemical informatics data from a vast repository
   - e.g. just search for any compound above
   - Has a graphical web interface and REST API
* (There are other computational chemistry REST APIs, including [Open Chemistry](https://doi.org/10.1186/s13321-017-0241-z), and [AFLOW-ML](https://doi.org/10.1016/j.commatsci.2018.03.075))


In [3]:
# dict mapping compound name to PubChemID
PubChemIDs = {
    'Methane': '297',
    'Benzene': '241',
    'Ethylammonium Iodide': '11116533'
}

The URL below is the compound 'API endpoint' which I can send a request to for properties of compounds given their PubChem CIDs

In [4]:
response = requests.get(('https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound'          # What type of entity? Compound
                         f'/cid/{",".join(PubChemIDs.values())}/'                      # Which compounds? These
                         'property/MolecularFormula,MolecularWeight,CanonicalSMILES/'  # Which properties? 
                         'JSON'),                                                      # Which format?
                        proxies=proxies)

In [5]:
response

<Response [200]>

In [6]:
response.json()

{'PropertyTable': {'Properties': [{'CID': 297,
    'MolecularFormula': 'CH4',
    'MolecularWeight': 16.043,
    'CanonicalSMILES': 'C'},
   {'CID': 241,
    'MolecularFormula': 'C6H6',
    'MolecularWeight': 78.11,
    'CanonicalSMILES': 'C1=CC=CC=C1'},
   {'CID': 11116533,
    'MolecularFormula': 'C2H8IN',
    'MolecularWeight': 173,
    'CanonicalSMILES': 'CCN.I'}]}}

## The ESCALATE REST API 

* Send data back and forth between ML, chemists, and laboratories

### Interactive List of all available ESCALATE endpoints

Simply navigating to http://localhost:8000/api will show you a browsable list of all available API endpoints. 

We can also view that list programmatically: 

In [7]:
base_url = 'http://localhost:8000/api'  # local dev server
response = requests.get(base_url)
response.json()

{'action': 'http://localhost:8000/api/action/',
 'actiondef': 'http://localhost:8000/api/actiondef/',
 'actionparameter': 'http://localhost:8000/api/actionparameter/',
 'actionparameterdefassign': 'http://localhost:8000/api/actionparameterdefassign/',
 'actor': 'http://localhost:8000/api/actor/',
 'billofmaterials': 'http://localhost:8000/api/billofmaterials/',
 'bomcompositematerial': 'http://localhost:8000/api/bomcompositematerial/',
 'bommaterial': 'http://localhost:8000/api/bommaterial/',
 'calculation': 'http://localhost:8000/api/calculation/',
 'calculationdef': 'http://localhost:8000/api/calculationdef/',
 'compositematerial': 'http://localhost:8000/api/compositematerial/',
 'compositematerialproperty': 'http://localhost:8000/api/compositematerialproperty/',
 'condition': 'http://localhost:8000/api/condition/',
 'conditioncalculationdefassign': 'http://localhost:8000/api/conditioncalculationdefassign/',
 'conditiondef': 'http://localhost:8000/api/conditiondef/',
 'experiment': '

### Logging into ESCALATE


To be able to view and post to all endpoints, I will create a log in session that is managed by a token. 

In [8]:
# demo login credentials
login_data = {
    'username': 'mtynes',
    'password': 'hello1world2'
}

r_login = requests.post(f'{base_url}/login', data=login_data)
token = r_login.json()['token']
token

'5a24fdc4fe15eb2152f63b8f1f1c1f6538ba0ee4'

In [9]:
token_header = {'Authorization': f'Token {token}'}

content_type_header = {'content-type': 'application/json'} # for most requests we'll want this header

This token will allow me to validate my identity for this session

### Simple helper functions for GET/POST

These functions will do some minimal URL generation and JSON response parsing on top of what is done by the `requests` library

In [10]:
def post_data(endpoint, data={}, headers={**token_header, **content_type_header}):
    """POST `data` to `endpoint`in ESCALATE API using `headers`
    
    return: (dict|requests.Response), bool
    """
    r = requests.post(f'{base_url}/{endpoint}/', 
                      data=json.dumps(data), 
                      headers=headers)
    print(r)
    if r.ok: 
        print('POST: OK, returning new resource dict')
        return r.json()
    print('POST: FAILED, returning response object')
    return r


def get_data(endpoint, data={}, headers={**token_header}):
    """Make GET request with `data` to `endpoint` in ESCALATE API using `headers`
    
    return: (dict|list|requests.Response), bool
    """
    r = requests.get(f'{base_url}/{endpoint}/', params=data, headers=headers)
    print(r)
    if r.ok: 
        print('GET: OK')
        
        resp_json = r.json()        
        
        # handle cases: one vs many results
        if resp_json.get('count') is None: # edge case: template edit
            return r.json()
        elif resp_json.get('count') == 1: 
            print('Found one resource, returning dict')
            return resp_json['results'][0]
        elif resp_json.get('count') >= 1: 
            print(f"Found {resp_json['count']} resources, returning list of dicts)")
            return r.json()['results']
        else:
            print('GET: FAILED, returning response object')
    return r

### GET all of materials defined in ESCALATE

In [11]:
r = get_data(endpoint='material')

<Response [200]>
GET: OK
Found 294 resources, returning list of dicts)


All of these materials would probably take up too much space in this notebook, lets look at the first one.

All lead iodide shares in this chemical formula, etc. Models are jsut 'what is is about these thigns that are all the same, as opposed to particular vials of gblthat has volumes, contanimanants, dates, masses. these all come from the object, but share in model. models dont contain provenance, are only abotu intenseive properties 

In [12]:
r[0]

{'url': 'http://localhost:8000/api/material/a7b37faf-4696-45d6-836f-cb6725d57bce/',
 'uuid': 'a7b37faf-4696-45d6-836f-cb6725d57bce',
 'edocs': [],
 'tags': [],
 'notes': [],
 'description': 'Gamma-Butyrolactone',
 'consumable': True,
 'composite_flg': False,
 'material_class': 'model',
 'actor_description': None,
 'status_description': 'active',
 'add_date': '2021-04-21T13:06:03.145949Z',
 'mod_date': '2021-04-21T13:06:10.239921Z',
 'actor': None,
 'status': 'http://localhost:8000/api/status/3dd5a6a9-bc62-4533-a4c9-ee2fc4c33c82/',
 'material_types': ['http://localhost:8000/api/materialtype/049035f6-191f-4e36-9eb0-3925ad386062/',
  'http://localhost:8000/api/materialtype/693b6398-2b88-4184-8da7-589e335e561e/',
  'http://localhost:8000/api/materialtype/4e006667-e855-4e28-8a76-a389e0f943a1/'],
 'property': []}

These are the fields available for materials. Notably we can associate a material with arbitrary properties and material types.

### POST a material property

#### Current property definitions in ESCALATE

ESCALATE supports user defined property definitions, these are the one's we're using

In [13]:
r = get_data('propertydef', 
             {'fields': ['description']} # we can select 'columns' with the fields parameter    
            )
r

<Response [200]>
GET: OK
Found 16 resources, returning list of dicts)


[{'description': 'particle-size {min, max}'},
 {'description': 'mesh {min, max}'},
 {'description': 'capacity'},
 {'description': 'cross-linkage %'},
 {'description': 'moisture % {min, max}'},
 {'description': 'Resin Class'},
 {'description': 'Functional group'},
 {'description': 'concentration_molarity'},
 {'description': 'manufacturer'},
 {'description': 'plate well count'},
 {'description': 'plate well location'},
 {'description': 'plate well robot order'},
 {'description': 'plate well volume {min, max}'},
 {'description': 'concentration'},
 {'description': 'molecular-weight'},
 {'description': 'Crystal Score'}]

#### Molecular Weight

In [14]:
mw_property_def = get_data('propertydef', 
                           {'description': 'molecular-weight', # find an entity that has a particular description
                            'fields': ['url', 'description', 'val_unit']})
mw_property_def

<Response [200]>
GET: OK
Found one resource, returning dict


{'url': 'http://localhost:8000/api/propertydef/eec6d26c-aa2b-4eba-8693-849c7c3c9ea8/',
 'description': 'molecular-weight',
 'val_unit': 'g/mol'}

Now we can associate an instance of this property with a material, say EthNH3I

In [15]:
ethylammonium_iodide = get_data('material', 
                                {'description': 'Ethylammonium Iodide',
                                 'fields':['url', 'description', 'property']})
ethylammonium_iodide

<Response [200]>
GET: OK
Found one resource, returning dict


{'url': 'http://localhost:8000/api/material/3aaa3be4-1158-49d2-851b-360e884f0803/',
 'description': 'Ethylammonium Iodide',
 'property': ['http://localhost:8000/api/property/f3d50829-e1e7-4e16-b989-26d1a6549975/']}

Note the empty property list above

Lets fill that in with the molecular weight from PubChem

In [16]:
response = requests.get('https://pubchem.ncbi.nlm.nih.gov/rest/pug/'+\
                        f'compound/cid/{PubChemIDs.get("Ethylammonium Iodide")}/'+\
                        'property/MolecularFormula,MolecularWeight,CanonicalSMILES/JSON', 
                       proxies=proxies)
pubchem_json = response.json()
eth_mw = pubchem_json['PropertyTable']['Properties'][0]['MolecularWeight']
eth_mw

173

In [17]:
r = post_data('materialproperty',
                  {'material': ethylammonium_iodide['url'],
                   'property_def': mw_property_def['url'],
                   'value': f"{eth_mw}"
                  }
                 )

<Response [201]>
POST: OK, returning new resource dict


And we've stored it!

In [18]:
get_data('material', 
         {'description':'Ethylammonium Iodide'})

<Response [200]>
GET: OK
Found one resource, returning dict


{'url': 'http://localhost:8000/api/material/3aaa3be4-1158-49d2-851b-360e884f0803/',
 'uuid': '3aaa3be4-1158-49d2-851b-360e884f0803',
 'edocs': [],
 'tags': [],
 'notes': [],
 'description': 'Ethylammonium Iodide',
 'consumable': True,
 'composite_flg': False,
 'material_class': 'model',
 'actor_description': None,
 'status_description': 'active',
 'add_date': '2021-04-21T13:06:03.145949Z',
 'mod_date': '2021-04-21T13:06:10.239921Z',
 'actor': None,
 'status': 'http://localhost:8000/api/status/3dd5a6a9-bc62-4533-a4c9-ee2fc4c33c82/',
 'material_types': ['http://localhost:8000/api/materialtype/049035f6-191f-4e36-9eb0-3925ad386062/',
  'http://localhost:8000/api/materialtype/693b6398-2b88-4184-8da7-589e335e561e/'],
 'property': ['http://localhost:8000/api/property/f3d50829-e1e7-4e16-b989-26d1a6549975/',
  'http://localhost:8000/api/property/08577f8d-3df7-4392-917b-84f04aa618c3/']}


* In practice we can use this functionality to store properties from any experiment or calculation. 

* We can also store metadata about where these values came from   
  (example to come in a further tutorial on tags, notes, edocs, calculations).

## Action definitions

* Just as we are free to define properties, we are free to define actions
* Current definition are what we've needed to specify human/robot instructions for current workflows

In [19]:
r = get_data('actiondef', {'fields': ['description', 'uuid', 'url']})
r

<Response [200]>
GET: OK
Found 7 resources, returning list of dicts)


[{'url': 'http://localhost:8000/api/actiondef/7a066a15-1484-479c-b9b1-558b3d1e9e0c/',
  'uuid': '7a066a15-1484-479c-b9b1-558b3d1e9e0c',
  'description': 'dispense'},
 {'url': 'http://localhost:8000/api/actiondef/e005b856-c20b-4b04-8811-0409ca1236a7/',
  'uuid': 'e005b856-c20b-4b04-8811-0409ca1236a7',
  'description': 'heat_stir'},
 {'url': 'http://localhost:8000/api/actiondef/61c9185b-c528-4404-a0f1-706669e80464/',
  'uuid': '61c9185b-c528-4404-a0f1-706669e80464',
  'description': 'heat'},
 {'url': 'http://localhost:8000/api/actiondef/e4eba1f7-9157-4ff2-891f-19e0f2a4237a/',
  'uuid': 'e4eba1f7-9157-4ff2-891f-19e0f2a4237a',
  'description': 'start_node'},
 {'url': 'http://localhost:8000/api/actiondef/61da8b7f-3084-49ec-b18a-f2472e078132/',
  'uuid': '61da8b7f-3084-49ec-b18a-f2472e078132',
  'description': 'end_node'},
 {'url': 'http://localhost:8000/api/actiondef/45c517ef-b33b-45eb-999b-d94bb8e96fde/',
  'uuid': '45c517ef-b33b-45eb-999b-d94bb8e96fde',
  'description': 'dispense_solid'},

#### Zooming in on the dispense action definition

In [20]:
get_data('actiondef',               
         {'description': 'dispense',  # which action def
          'expand': 'parameter_def',  # sub dictionary to expand
          })

<Response [200]>
GET: OK
Found one resource, returning dict


{'url': 'http://localhost:8000/api/actiondef/7a066a15-1484-479c-b9b1-558b3d1e9e0c/',
 'uuid': '7a066a15-1484-479c-b9b1-558b3d1e9e0c',
 'edocs': [],
 'tags': [],
 'notes': [{'url': 'http://localhost:8000/api/note/6cee29c6-baae-4fb7-a9d2-809b307f0cbb/',
   'uuid': '6cee29c6-baae-4fb7-a9d2-809b307f0cbb',
   'notetext': 'source material = material to be dispensed, destination material = plate well',
   'add_date': '2021-04-21T13:06:07.596763Z',
   'mod_date': '2021-04-21T13:06:07.596763Z',
   'actor_description': 'Mike Tynes',
   'ref_note_uuid': '7a066a15-1484-479c-b9b1-558b3d1e9e0c',
   'actor': 'http://localhost:8000/api/actor/73dea3b1-9908-4b27-b117-0c082595eb87/'}],
 'parameter_def': [{'url': 'http://localhost:8000/api/parameterdef/b1188365-c04d-47f9-b5ee-57f0af45c5c3/',
   'uuid': 'b1188365-c04d-47f9-b5ee-57f0af45c5c3',
   'edocs': [],
   'tags': [],
   'notes': [],
   'description': 'volume',
   'default_val': {'value': 0.0, 'unit': 'mL', 'type': 'num'},
   'unit_type': 'volume',
  

### Actions + Materials = Experiment Template 

* Experiment template = the form of an experiment that I wish to re-use, varying material choices and process parameters

In [21]:
experiment_templates = get_data('experimenttemplate', 
                               {'fields':['description', 'url']})

<Response [200]>
GET: OK
Found 4 resources, returning list of dicts)


In [22]:
experiment_templates

[{'url': 'http://localhost:8000/api/experimenttemplate/aa347bfd-a7b3-4689-a933-1f0739ca21f0/',
  'description': 'liquid_solid_extraction'},
 {'url': 'http://localhost:8000/api/experimenttemplate/e4bb37ec-50a5-41b2-b888-e4c0f2e78a3c/',
  'description': 'resin_weighing'},
 {'url': 'http://localhost:8000/api/experimenttemplate/fa486071-47d9-4a17-8f91-3d71a6be1ddd/',
  'description': 'perovskite_demo'},
 {'url': 'http://localhost:8000/api/experimenttemplate/1d843469-5903-4265-bfd4-52392ab448f4/',
  'description': 'test_wf_1'}]

Click on perovskite demo link and note 2 main nested fields: 
* Bill of materials = the initial materials for the perovskite workflow
* Workflow = the set of actions that combine these materials into perovskite crystal trials

In [23]:
perovskite_template = get_data('experimenttemplate',
                              {'description': 'test_wf_1', 
                              'expand': 'workflow' # expand the workflow subdictionary
                              })

<Response [200]>
GET: OK
Found one resource, returning dict


In [24]:
perovskite_template

{'url': 'http://localhost:8000/api/experimenttemplate/1d843469-5903-4265-bfd4-52392ab448f4/',
 'uuid': '1d843469-5903-4265-bfd4-52392ab448f4',
 'edocs': [],
 'tags': [],
 'notes': [],
 'bill_of_materials': [{'url': 'http://localhost:8000/api/billofmaterials/a522c44d-7d3d-455c-973b-0983d7802b10/',
   'uuid': 'a522c44d-7d3d-455c-973b-0983d7802b10',
   'edocs': [],
   'tags': [],
   'notes': [],
   'bom_material': ['http://localhost:8000/api/bommaterial/96743e98-1d50-4728-8644-01a06c4209f0/',
    'http://localhost:8000/api/bommaterial/4f7a29dd-1d22-44de-b1ee-f80e0f6774ba/',
    'http://localhost:8000/api/bommaterial/dd8ee50d-f0a8-4d5a-b940-d7bcd264a06f/',
    'http://localhost:8000/api/bommaterial/314f76d2-8815-4e09-826e-102298c2df78/',
    'http://localhost:8000/api/bommaterial/dde062ce-1227-46a7-a2c6-9cd032958c72/'],
   'description': 'Test WF1 Materials',
   'experiment_description': 'test_wf_1',
   'add_date': '2021-04-21T13:06:10.378710Z',
   'mod_date': '2021-04-21T13:06:10.378710Z'

## Bill of Materials

The initial materials: think list of materials in methods section of paper

These are the Bill of Materials' Material entries for this experiment: 

In [25]:
get_data('bommaterial', {'bom':perovskite_template['bill_of_materials'][0]['uuid'], 
                        'fields':['description']})

<Response [200]>
GET: OK
Found 5 resources, returning list of dicts)


[{'description': 'Acid'},
 {'description': 'Solvent'},
 {'description': 'Stock A'},
 {'description': 'Stock B'},
 {'description': 'Plate'}]

In [26]:
get_data('compositematerial', {'composite_description__startswith':'Stock', 
                               'fields': ['composite_description',
                                          'component_description']})

<Response [200]>
GET: OK
Found 7 resources, returning list of dicts)


[{'composite_description': 'Stock A',
  'component_description': 'Lead Diiodide'},
 {'composite_description': 'Stock A',
  'component_description': 'Ethylammonium Iodide'},
 {'composite_description': 'Stock A',
  'component_description': 'Gamma-Butyrolactone'},
 {'composite_description': 'Stock B',
  'component_description': 'Ethylammonium Iodide'},
 {'composite_description': 'Stock B',
  'component_description': 'Gamma-Butyrolactone'},
 {'composite_description': 'Stock FAH',
  'component_description': 'Formic Acid'},
 {'composite_description': 'Stock FAH', 'component_description': 'Water'}]

#### We could drill further to get the concentrations, properties, etc

### Workflows: Logical groups of actions

Each of these contains a set of parameters that we can edit

In [27]:
perovskite_demo = get_data('experimenttemplate',
         {'description': 'perovskite_demo',
          'expand': 'workflow'})
[wf['description'] for wf in perovskite_demo['workflow']]

<Response [200]>
GET: OK
Found one resource, returning dict


['Perovskite Demo: Preheat Plate',
 'Perovskite Demo: Prepare Stock A',
 'Perovskite Demo: Prepare Stock B',
 'Perovskite Demo: Dispense Solvent',
 'Perovskite Demo: Dispense Stock A',
 'Perovskite Demo: Dispense Stock B',
 'Perovskite Demo: Dispense Acid Vol 1',
 'Perovskite Demo: Heat Stir 1',
 'Perovskite Demo: Dispense Acid Vol 2',
 'Perovskite Demo: Heat Stir 2',
 'Perovskite Demo: Heat']

In [28]:
dispense_solvent_wf = perovskite_demo['workflow'][3] # pull out a workflow
example_steps = dispense_solvent_wf['step'][4]     # pull out some steps
example_steps

{'url': 'http://localhost:8000/api/workflowstep/e8f3ceec-152a-4ee3-889e-f86b550aa8d6/',
 'uuid': 'e8f3ceec-152a-4ee3-889e-f86b550aa8d6',
 'edocs': [],
 'tags': [],
 'notes': [],
 'workflow_object': 'http://localhost:8000/api/workflowobject/04dd3f83-6a7c-44d8-ab71-de416ac0ffdf/',
 'workflow_description': 'Perovskite Demo: Dispense Solvent',
 'parent_object_type': 'action',
 'parent_object_description': 'Perovskite Demo: Dispense Solvent: Solvent -> Plate: Plate well#: A4',
 'parent_path': None,
 'conditional_val': None,
 'conditional_value': None,
 'status_description': None,
 'add_date': '2021-04-21T13:06:12.521845Z',
 'mod_date': '2021-04-21T13:06:12.521845Z',
 'workflow': 'http://localhost:8000/api/workflow/e24220d4-a83d-46d3-baa4-11f89a2f34fc/',
 'parent': 'http://localhost:8000/api/workflowstep/440dd4b3-2311-41a1-8638-953dbedcfa29/',
 'status': None}

## Creating a new workflow from a template

If I want create an instance experiment from a template I: 

In [29]:
editable_template = get_data(
    # template endpoint  /        template ID      / create an instance of this template 
    f'experimenttemplate/{perovskite_demo["uuid"]}/create'
)

<Response [200]>
GET: OK


In [30]:
editable_template

{'experiment_name': '',
 'material_parameters': [{'material_name': 'Stock FAH',
   'value': 'http://localhost:8000/api/inventorymaterial/bf74ac72-ea82-423c-9214-63a68aee5e4c/'},
  {'material_name': 'Neat GBL',
   'value': 'http://localhost:8000/api/inventorymaterial/43b80e1d-cbed-4985-ae45-175b9c7a3fc8/'},
  {'material_name': 'Ethylammonium Iodide',
   'value': 'http://localhost:8000/api/inventorymaterial/30733f43-252d-42c6-8cf0-bc80655432cc/'},
  {'material_name': 'Lead Diiodide',
   'value': 'http://localhost:8000/api/inventorymaterial/f2da499c-f48b-4752-a777-c2359b20a292/'},
  {'material_name': 'Wf1 Plate',
   'value': 'http://localhost:8000/api/inventorymaterial/e366f483-1393-4da5-b52b-6e9710070280/'},
  {'material_name': 'Tube: 5mL',
   'value': 'http://localhost:8000/api/inventorymaterial/aeb511b7-5a42-4f89-9fb0-a128f172a5f3/'},
  {'material_name': 'Tube: 5mL',
   'value': 'http://localhost:8000/api/inventorymaterial/aeb511b7-5a42-4f89-9fb0-a128f172a5f3/'}],
 'experiment_paramete

Can also give arrays of values over 96 well plates

In [31]:
editable_template['experiment_name'] = 'test_perovskite_instance'

Suppose I edit this json then I can post the new template to the server

In [32]:
resp = post_data(
    f'experimenttemplate/{perovskite_demo["uuid"]}/create',
    editable_template
)

<Response [200]>
POST: OK, returning new resource dict


In [33]:
resp

{'new_experiment_created': 'http://localhost:8000/api/experiment/235a27de-ed75-46c2-a2e7-c3b94cb226e2/'}

This experiment then appears in the experiment queue.   

http://localhost:8000/experiment_list

The experimentalist is notified, can download relevant robot input and upload observed values through forms

In [34]:
%%time
new_experiment_json = get_data('experiment/' + resp['new_experiment_created'].split('/')[-2], 
                                {'expand': 'workflow.step.workflow_object.action.parameter'}) # expanding deeply nested fields is somewhat slow

<Response [200]>
GET: OK
CPU times: user 4.79 ms, sys: 1.88 ms, total: 6.67 ms
Wall time: 2.41 s


In [35]:
experiment_json = [new_experiment_json]

In [36]:
def experiment_json_to_df(experiment_json):
    result = []
    for e in experiment_json: 
        for workflow in e['workflow']:
            if 'Dispense' not in workflow['description']:
                    continue
            for step in workflow['step']: 
                action = step['workflow_object']['action']
                for parameter in action['parameter']:
                    result.append(
                        dict(
                            experiment_url         = e['url'],
                            experiment_id          = e['url'].split('/')[-2],
                            action_source          = action['source_material_description'],
                            action_dest            = action['destination_material_description'],
                            action_parameter       = parameter['parameter_def_description'],
                            action_parameter_value = float(parameter['parameter_val_nominal']['value']),
                            action_parameter_unit  = parameter['parameter_val_nominal']['unit']
                            )
                        )
    return pd.DataFrame(result)

In [37]:
results = experiment_json_to_df(experiment_json)

In [38]:
results

,experiment_url,experiment_id,action_source,action_dest,action_parameter,action_parameter_value,action_parameter_unit
0,http://localhost:8000/api/experiment/235a27de-...,235a27de-ed75-46c2-a2e7-c3b94cb226e2,Solvent,Plate: Plate well#: A1,volume,0.0,mL
1,http://localhost:8000/api/experiment/235a27de-...,235a27de-ed75-46c2-a2e7-c3b94cb226e2,Solvent,Plate: Plate well#: A6,volume,0.0,mL
2,http://localhost:8000/api/experiment/235a27de-...,235a27de-ed75-46c2-a2e7-c3b94cb226e2,Solvent,Plate: Plate well#: A5,volume,0.0,mL
3,http://localhost:8000/api/experiment/235a27de-...,235a27de-ed75-46c2-a2e7-c3b94cb226e2,Solvent,Plate: Plate well#: A4,volume,0.0,mL
4,http://localhost:8000/api/experiment/235a27de-...,235a27de-ed75-46c2-a2e7-c3b94cb226e2,Solvent,Plate: Plate well#: A3,volume,0.0,mL
5,http://localhost:8000/api/experiment/235a27de-...,235a27de-ed75-46c2-a2e7-c3b94cb226e2,Solvent,Plate: Plate well#: A2,volume,0.0,mL
6,http://localhost:8000/api/experiment/235a27de-...,235a27de-ed75-46c2-a2e7-c3b94cb226e2,Stock A,Plate: Plate well#: A1,volume,0.0,mL
7,http://localhost:8000/api/experiment/235a27de-...,235a27de-ed75-46c2-a2e7-c3b94cb226e2,Stock A,Plate: Plate well#: A6,volume,0.0,mL
8,http://localhost:8000/api/experiment/235a27de-...,235a27de-ed75-46c2-a2e7-c3b94cb226e2,Stock A,Plate: Plate well#: A5,volume,0.0,mL
9,http://localhost:8000/api/experiment/235a27de-...,235a27de-ed75-46c2-a2e7-c3b94cb226e2,Stock A,Plate: Plate well#: A4,volume,0.0,mL


In [39]:
results = results.pivot_table(index=['experiment_id', 'action_dest'], 
                      columns=['action_source'], 
                      values='action_parameter_value')
results

action_source                                                Acid  Solvent  \
experiment_id                        action_dest                             
235a27de-ed75-46c2-a2e7-c3b94cb226e2 Plate: Plate well#: A1   0.0      0.0   
                                     Plate: Plate well#: A2   0.0      0.0   
                                     Plate: Plate well#: A3   0.0      0.0   
                                     Plate: Plate well#: A4   0.0      0.0   
                                     Plate: Plate well#: A5   0.0      0.0   
                                     Plate: Plate well#: A6   0.0      0.0   

action_source                                                Stock A  Stock B  
experiment_id                        action_dest                               
235a27de-ed75-46c2-a2e7-c3b94cb226e2 Plate: Plate well#: A1      0.0      0.0  
                                     Plate: Plate well#: A2      0.0      0.0  
                                     Plate: Plate well#: A3      0.0      0.0  
                                     Plate: Plate well#: A4      0.0      0.0  
                                     Plate: Plate well#: A5      0.0      0.0  
                                     Plate: Plate well#: A6      0.0      0.0

In [40]:
crystal_scores = get_data('measure', 
                          {'measuredef':
                                (get_data('measuredef', 
                                          {'description': 'crystal_score'})['url']
                                )
                          })                        
results['crystal_score'] = crystal_scores['measure_value']['value']

<Response [200]>
GET: OK
Found one resource, returning dict
<Response [200]>
GET: OK
Found one resource, returning dict


In [41]:
results

action_source                                                Acid  Solvent  \
experiment_id                        action_dest                             
235a27de-ed75-46c2-a2e7-c3b94cb226e2 Plate: Plate well#: A1   0.0      0.0   
                                     Plate: Plate well#: A2   0.0      0.0   
                                     Plate: Plate well#: A3   0.0      0.0   
                                     Plate: Plate well#: A4   0.0      0.0   
                                     Plate: Plate well#: A5   0.0      0.0   
                                     Plate: Plate well#: A6   0.0      0.0   

action_source                                                Stock A  Stock B  \
experiment_id                        action_dest                                
235a27de-ed75-46c2-a2e7-c3b94cb226e2 Plate: Plate well#: A1      0.0      0.0   
                                     Plate: Plate well#: A2      0.0      0.0   
                                     Plate: Plate well#: A3      0.0      0.0   
                                     Plate: Plate well#: A4      0.0      0.0   
                                     Plate: Plate well#: A5      0.0      0.0   
                                     Plate: Plate well#: A6      0.0      0.0   

action_source                                                crystal_score  
experiment_id                        action_dest                            
235a27de-ed75-46c2-a2e7-c3b94cb226e2 Plate: Plate well#: A1              1  
                                     Plate: Plate well#: A2              1  
                                     Plate: Plate well#: A3              1  
                                     Plate: Plate well#: A4              1  
                                     Plate: Plate well#: A5              1  
                                     Plate: Plate well#: A6              1

### Current Limitations

* Some parts of API still are 'high entropy' (e.g. measure)
* Ditto for some portions of UI
* REST is slow for large transfers